In [15]:
%%capture
%pip install openai
%pip install python-dotenv
%pip install matplotlib
%pip install pillow
%pip install openai==1.65.2
%pip install tiktoken 
%pip install pandas

In [16]:
# Importing the necessary Python libraries
import IPython
import time
import os
import openai
import dotenv
from openai import AzureOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown
from azure.core.credentials import AzureKeyCredential
import sqlite3
import re

In [17]:
def run_query(sql_query: str, user_id: str):
    #connect database
    conn = sqlite3.connect("phase-2.db")
    cursor = conn.cursor()

    # format the sql correctly
    if "WHERE" in sql_query.upper():
        if "uid =" not in sql_query:
            sql_query += " AND uid = ?"
            params = (user_id,)
        else:
            params = ()
    else:
        sql_query += " WHERE uid = ?"
        params = (user_id,)

    cursor.execute(sql_query, params)
    rows = cursor.fetchall()
    conn.close()
    return rows

def main():
    deployment = "gpt-4.1-mini"
    user_uid = "55555555"

    client = AzureOpenAI(
        azure_endpoint="https://aisdevelopment.openai.azure.com/",
        api_key="DTyQG79lV7tPjYYFAB9sGzYe8MkQSrdLsosDYlUEIqAjNQ9NDtZZJQQJ99BFACYeBjFXJ3w3AAABACOGBm6d",
        api_version="2024-12-01-preview",
    )

    text = input("\nEnter a question:\n")

    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {"role": "system", 
            "content": """You are an AI assistant that answers questions about a university admissions system.
                        You have access to the schema and should generate SQL queries when necessary, but always filter by uid = '{user_uid}'.
                        Respond clearly and concisely with answers based on the user's data.
                        Never expose or guess data from other users.
                        Refrence the sql schema provided before writing any sql queries to ensure you properly refrence the correct tables and columns.
                        When you need to fetch data, feel free to provide and use the exact SQL query you want to run by calling
                        the function run_query(sql_query, user_uid).
                        Think the question through before answering and solve it step by step."""
            },
            {"role": "user", "content": text},
        ],
        extra_body={
            "data_sources": [
                {
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": "https://registrarsearch.search.windows.net",
                        "index_name": "rag-1750953593343",
                        "authentication": {
                            "type": "api_key",
                            "key": "VwskNOUDEkwKfLuoHPUzZiqdYRJA3KyM7Dq51U7EFWAzSeCiE8XC",
                        },
                    },
                }
            ],
        },
    )

    result = completion.choices[0].message.content
    print("\nAI says:\n", result)

    # Naive example: Extract SQL query if it exists between ```sql ... ``` blocks or from prompt
    if result is not None:
        sql_match = re.search(r"```sql\s*(SELECT.+?)```", result, re.IGNORECASE | re.DOTALL)
        if sql_match:
            sql_query = sql_match.group(1).strip()
            print("\nRunning SQL query:\n", sql_query)
            rows = run_query(sql_query, user_uid)
            print("\nSQL query results:")
            for row in rows:
                print(row)
    else:
        print("\nNo SQL query detected.")

if __name__ == "__main__":
    main()



AI says:
 To list all the classes offered, we need to retrieve the course information from the course catalog and the schedule of classes. The classes include details such as department, course number, title, credit hours, semester, day, and time.

Based on the schema and documents, the relevant information is stored in tables related to courses and schedules. I will write a query to list all classes offered for your user ID.

Here is the SQL query to get the list of all classes offered:

```sql
SELECT DISTINCT
    c.department,
    c.course_number,
    c.title,
    c.credit_hours,
    s.semester,
    s.day,
    s.time
FROM
    courses c
JOIN
    schedule s ON c.department = s.department AND c.course_number = s.course_number
WHERE
    uid = '{user_uid}';
```

I will now run this query for you.

Running SQL query:
 SELECT DISTINCT
    c.department,
    c.course_number,
    c.title,
    c.credit_hours,
    s.semester,
    s.day,
    s.time
FROM
    courses c
JOIN
    schedule s ON c.dep

OperationalError: no such column: c.department